In [3]:
import torch
import zipfile
import numpy as np

device ='cpu'  #cuda 会报错，见6.0


def load_data_jay_lyrics():
    with zipfile.ZipFile('Datasets/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

In [4]:
def one_hot(x,n_class,dtype=torch.float32):
    # x batchsize  output batchsize,vocab_size 一个矩阵，分numsteps处理 vocab_size即n_class
    x = x.long()
    res = torch.zeros(x.shape[0],n_class,dtype=dtype,device=device)
    res.scatter_(1,x.view(-1,1),1)  #最后一个1是填充1，可以改任意整数值，或者矩阵
    return res

#torch.gather(input, dim, index, out=None) → Tensor 聚集操作
#torch.Tensor scatter_(dim, index, src) → Tensor 分散操作 

x = torch.tensor([0, 2])
print(one_hot(x, 6))

def to_onehot(X,n_class): # for epoch，分批次转化
    #X shape (batchsize,numsteps)  output numsteps个 batchsize,vocab_size的矩阵
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])]



tensor([[1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.]])


In [5]:
import random
def data_iter_random(corpus_indices,batch_size,num_steps,device=None):
    num_examples = (len(corpus_indices) - 1)//num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    #print(example_indices)
    random.shuffle(example_indices)
    
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(device)
    
    for i in range(epoch_size):
        i = i * batch_size
        batch_indices = example_indices[i:i+batch_size]
        X = [corpus_indices[j*num_steps:j*num_steps + num_steps]  for j in batch_indices]
        Y = [corpus_indices[j*num_steps+1:j*num_steps + num_steps+1]  for j in batch_indices]
        yield torch.tensor(X,dtype=torch.float32,device=device),torch.tensor(Y,dtype=torch.float32,device=device)
my_seq = list(range(31))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6,device='cpu'):
    print('X: ', X, '\nY:', Y, '\n')
    

def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    batch_len = len(corpus_indices)//batch_size
    indices = corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    #print(indices)
    epoch_size =(batch_len -1)//num_steps
    for i in range(epoch_size):
        i = i*num_steps
        X = indices[:,i:i+num_steps]  #多维数组切片
        Y = indices[:,i+1:i+num_steps+1]
        yield X,Y

X:  tensor([[12., 13., 14., 15., 16., 17.],
        [18., 19., 20., 21., 22., 23.]]) 
Y: tensor([[13., 14., 15., 16., 17., 18.],
        [19., 20., 21., 22., 23., 24.]]) 

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10., 11.]]) 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [ 7.,  8.,  9., 10., 11., 12.]]) 



In [6]:
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device = device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
     # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return torch.nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])
params = get_params()

In [7]:
def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros(batch_size,num_hiddens,device=device),) #元组，其实可以不用，示例在多个元素的事后如何处理

def rnn(inputs,state,params):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X,W_xh)+torch.matmul(H,W_hh)+b_h)
        Y = torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,
                num_hiddens,vocab_size,device,idx_to_char,char_to_idx):
    state = init_rnn_state(1,num_hiddens,device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        X = to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        (Y,state) = rnn(X,state,params)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join( idx_to_char[i] for i in output)

predict_rnn('分开',10,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx)

'分开家绕宠女若酒养方愿潮'

In [12]:
#使用困惑度，perplexity评价语言模型的好坏，困惑度是对交叉熵损失函数做指数运算后得到的值
import math
import time
#import data1  # ipynb文件，无法当模块导入
def grad_clipping(params,theta,device):  #防止梯度爆炸
    norm = torch.tensor([0.0],device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()  #裁剪的是梯度值
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *=(theta/norm)
            
def sgd(params,lr,batch_size):  #更新的时候为什么 除以256？？？
    for param in params:
        param.data-=param.grad*lr/batch_size
            
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,device,corpus_indices,
                          idx_to_char,char_to_idx,is_random_iter,num_epochs,num_steps,lr,clipping_theta,
                          batch_size,pred_period,pred_len,prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = torch.nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter: #如使用相邻采样，一次循环初始化一次状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum,n,start = 0.0,0,time.time()
        data_iter = data_iter_fn(corpus_indices,batch_size,num_steps,device)
        for X,Y in data_iter: #X 
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  
            # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach_()
            
            inputs = to_onehot(X,vocab_size)
            (outputs,state) = rnn(inputs,state,params) #inputs numsteps个 batchsize,vocab_size的矩阵 
            outputs = torch.cat(outputs, dim=0)
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(outputs, y.long())
            
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度 
            #不裁剪梯度，困惑度超出范围  按交叉熵计算的话，也能得到结果
            sgd(params, lr, 1)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (  #perplexity exp(l_sum/n)
                epoch + 1, (l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [11]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 4.282180, time 0.38 sec
 - 分开 我想要的可写女人 坏成的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏
 - 不分开 快使在美 我已定的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可
epoch 100, perplexity 2.262808, time 0.38 sec
 - 分开 一颗用双截 我只往有抽瘦 太色入秋的牛肉 我说店小二 三两银够不够 景色入秋的牛肉 我说店小二 三
 - 不分开吗 我有好这生瘦 我知我有你 是你都红鸦球 一场正 一直走著我在天 一壶就酒濡汉我 想要你的黑色默默
epoch 150, perplexity 1.038553, time 0.41 sec
 - 分开 一颗两颗三颗四颗 连成线背著背默默许下心  我的世界已狂的封斑 一切内明演 会不人开 在小村外的溪
 - 不分开期 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我已经这节奏 后知后觉 
epoch 200, perplexity 0.466650, time 0.37 sec
 - 分开 有一人美昏 大静躺在抽屉 它所拥有见只离做 我想大声宣布 对你依依不舍 连隔都邻居都猜轻叹息 太多
 - 不分开扫 我叫你爸 你打我妈 这样对吗干嘛这样 何必让 牵泪眼 看么吐纳手心在 干什么 干什么 气吸丹田手
epoch 250, perplexity 0.260281, time 0.33 sec
 - 分开 沙愿到不多 有唱都红说喝水也能活 脑袋瓜有一点秀逗 猎物死了它比谁都难过 印地安斑鸠 会学人开口 
 - 不分开扫 我叫你爸 你打我妈 这样对吗干嘛这样 何必让酒牵鼻子走 瞎 说底睡写斯 周杰伦哭 泥家村外的溪边


In [8]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 61.895668, time 0.32 sec
 - 分开 我想要的可写 我有要这 我有了的可写 我有要这 我想了的可爱 我有要这 我想了的可爱 我有要这 我
 - 不分开 我想要的可爱 我有要这 我有了的可写 我有要这 我想了的可爱 我有要这 我想了的可爱 我有要这 我
epoch 100, perplexity 7.543522, time 0.32 sec
 - 分开 一子我 别怪我 说你怎么 对头一碗热  没有你在我有多烦熬多难熬  穿过云层 我试多努恼  没有你
 - 不分开只 你已经很你我 不知不觉 我已了这节奏 后知后觉 又过了一个秋 哼知不觉 快使用双截棍 哼哼哈兮 
epoch 150, perplexity 2.150913, time 0.32 sec
 - 分开 每候的美我面起 从发现迷的路土 不会骑河柳的胖女  穿在我遇坦堡你的家不有 一个苏美年圈出土 我以
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 200, perplexity 1.349285, time 0.32 sec
 - 分开 你说啊 你怎了话阳光 不管人哪里都 它在许愿安里 平呆了我 说你说 分数怎么停留 一直在停留 谁让
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 250, perplexity 1.155451, time 0.38 sec
 - 分开 你说啊 你怎载著阳手 不管到哪医都是说天 蝴蝶自在飞 花也布满天 一朵一朵因你而香 试图让夕阳飞翔
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 后知后觉 迷迷蒙蒙 你我的懂 疗了梦纵 隐隐作用
